In [1]:
import collections
import math
import numpy as np
import pandas as pd
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import stirling
import matplotlib.pyplot as plt 
%matplotlib notebook

Calculates number of set partitions given an integer n and a integer partition p

In [2]:
def calc_set_partitions(n,p):
    num = math.factorial(n)
    den = 1
    mul = 0
    for i in p.keys():
        m = p[i]
        den = den * pow(math.factorial(i), m) * math.factorial(m)
    frac = num / den
    return frac

find_choice calculates the correct choice of column size and value given a partition and the prior probability distribution represented in a dictionary

In [ ]:
def find_choice(p, prior):
    # generates table with type probability, adds, normalizes
    # generates df with sum of normalized probability where value = 'a', col = 1, etc.
    # maxes df
    # returns choice eg ['a', '1', 'prob(a == 1)']
    # returns prob(choice)
    print('todo')
    return 1

In [ ]:
def choose_g_post_vuln(n, k, Prior):
    g_sum = 0
    for p in partitions(n, m=k):  
        num = calc_set_partitions(n,p)
        frac = find_choice(p, Prior)
        val = num * frac # check
        g_sum = g_sum + val
    return g_sum